<a href="https://colab.research.google.com/github/warwavn/Kaggle-competition/blob/main/Temp_prediction_rndfrtregressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
for filename in uploaded.keys():
    df = pd.read_csv(filename)
    print(f"Loaded {filename} with shape {df.shape}")

Saving test.csv to test.csv
Loaded test.csv with shape (4091, 19)


In [2]:
import pandas as pd

train = pd.read_csv("train.csv")
train.head()

,id,mac,station_name,tambon_code,tambon_namt,amphur_code,amphur_namt,province_code,province_namt,latitude,longitude,time,humid,light,pm10,pm2.5,rainfall,wind_direct,wind_speed,temp
0,0,3C71BF164F90,โรงเรียนท่าข้ามวิทยา,860206,ท่าข้าม,8602,ท่าแซะ,86,ชุมพร,10.579849,99.113146,2022-07-08 19:00:00+07:00,89.9,5.0,6.0,5.0,0.0,180.0,6.1,26.925
1,1,3C71BF1B1E28,โรงเรียนสรรพวิทยาคม,630601,แม่สอด,6306,แม่สอด,63,ตาก,16.712990,98.573417,2022-07-18 00:00:00+07:00,80.8,0.0,8.0,7.0,0.0,135.0,1.4,27.250
2,2,30AEA4F7AE44,โรงเรียนสามเสนนอก(ประชาราษฎร์อนุกูล),102601,ดินแดง,1026,ดินแดง,10,กรุงเทพมหานคร,13.777972,100.569662,2022-06-19 11:00:00+07:00,60.8,51.0,NaN,NaN,0.0,270.0,3.9,34.075
3,3,3C71BF1B1E28,โรงเรียนสรรพวิทยาคม,630601,แม่สอด,6306,แม่สอด,63,ตาก,16.712990,98.573417,2022-06-18 03:00:00+07:00,87.1,0.0,6.0,5.0,0.0,135.0,0.0,26.500
4,4,807D3AF57920,บ้านนาสะแบง_2,380704,นาสะแบง,3807,ศรีวิไล,38,บึงกาฬ,18.142499,103.806521,2022-07-28 04:00:00+07:00,86.4,3.0,3.0,1.0,0.0,22.5,0.0,28.600


In [16]:
# Get a detailed summary of missing values
missing_summary = train.isnull().sum().reset_index()
missing_summary.columns = ['Column', 'Missing Values']
missing_summary = missing_summary[missing_summary['Missing Values'] > 0]
print(missing_summary)


   Column  Missing Values
14   pm10            4752
15  pm2.5            4752


In [9]:
pmchk2 = train.groupby('tambon_namt')['pm2.5'].mean()
print("pm 2.5 by tambon:")
print(pmchk2)

pmchk10 = train.groupby('tambon_namt')['pm10'].mean()
print("pm 10 by tambon:")
print(pmchk10)

pm 2.5 by tambon:
tambon_namt
กำปัง            0.135135
ดินแดง           1.511962
ท่าข้าม          5.980000
นาสะแบง          1.711823
บักดอง           7.812658
สถาน            11.465378
หนองสูงเหนือ     0.000000
แม่สอด           8.771405
Name: pm2.5, dtype: float64
pm 10 by tambon:
tambon_namt
กำปัง            1.135135
ดินแดง           2.799043
ท่าข้าม          6.964000
นาสะแบง          3.244663
บักดอง           8.871730
สถาน            12.600644
หนองสูงเหนือ     1.000000
แม่สอด           9.874480
Name: pm10, dtype: float64


In [3]:
pm25_tambon_means = train.groupby('tambon_namt')['pm2.5'].transform('mean')
train['pm2.5'].fillna(pm25_tambon_means, inplace=True)

pm10_tambon_means = train.groupby('tambon_namt')['pm10'].transform('mean')
train['pm10'].fillna(pm10_tambon_means, inplace=True)

In [4]:
test = pd.read_csv("test.csv")
test.head()

,id,mac,station_name,tambon_code,tambon_namt,amphur_code,amphur_namt,province_code,province_namt,latitude,longitude,time,humid,light,pm10,pm2.5,rainfall,wind_direct,wind_speed
0,9543,3C71BF1B1E28,โรงเรียนสรรพวิทยาคม,630601,แม่สอด,6306,แม่สอด,63,ตาก,16.712990,98.573417,2022-06-04 09:00:00+07:00,72.2,84.0,3.0,2.0,0.0,90.0,4.6
1,9544,3C71BF164F90,โรงเรียนท่าข้ามวิทยา,860206,ท่าข้าม,8602,ท่าแซะ,86,ชุมพร,10.579849,99.113146,2022-06-24 15:00:00+07:00,76.8,66.0,NaN,NaN,0.0,225.0,1.5
2,9545,807D3AF57920,บ้านนาสะแบง_2,380704,นาสะแบง,3807,ศรีวิไล,38,บึงกาฬ,18.142499,103.806521,2022-06-15 07:00:00+07:00,90.4,27.0,4.0,3.0,2.3,0.0,0.0
3,9546,3C71BF18CEA4,โรงเรียนหนองสูงสามัคคีวิทยา_2,490706,หนองสูงเหนือ,4907,หนองสูง,49,มุกดาหาร,16.494229,104.350891,2022-05-12 04:00:00+07:00,97.9,0.0,NaN,NaN,0.0,45.0,0.0
4,9547,3C71BF164F90,โรงเรียนท่าข้ามวิทยา,860206,ท่าข้าม,8602,ท่าแซะ,86,ชุมพร,10.579849,99.113146,2022-06-04 12:00:00+07:00,91.3,43.0,NaN,NaN,0.0,45.0,1.2


In [5]:
# Convert the datetime column to pandas datetime object
train['time'] = pd.to_datetime(train['time'])

# Extract year, month, and time period
train['year'] = train['time'].dt.year
train['month'] = train['time'].dt.month

# Define a function to categorize the time of day
def categorize_time(hour):
    if 0 <= hour < 6:
        return  1 #'Night'
    elif 6 <= hour < 12:
        return 2 #'Morning'
    elif 12 <= hour < 18:
        return 3 #'Afternoon'
    else:
        return 4 #'Evening'

train['time_period'] = train['time'].dt.hour.apply(categorize_time)

In [6]:
train.drop(['station_name', 'tambon_namt','amphur_namt','province_namt','time'], axis=1, inplace=True)

In [45]:
train

,id,mac,tambon_code,amphur_code,province_code,latitude,longitude,humid,light,pm10,pm2.5,rainfall,wind_direct,wind_speed,temp,year,month,time_period
0,0,3C71BF164F90,860206,8602,86,10.579849,99.113146,89.9,5.0,6.000000,5.000000,0.0,180.0,6.1,26.925,2022,7,4
1,1,3C71BF1B1E28,630601,6306,63,16.712990,98.573417,80.8,0.0,8.000000,7.000000,0.0,135.0,1.4,27.250,2022,7,1
2,2,30AEA4F7AE44,102601,1026,10,13.777972,100.569662,60.8,51.0,2.799043,1.511962,0.0,270.0,3.9,34.075,2022,6,2
3,3,3C71BF1B1E28,630601,6306,63,16.712990,98.573417,87.1,0.0,6.000000,5.000000,0.0,135.0,0.0,26.500,2022,6,1
4,4,807D3AF57920,380704,3807,38,18.142499,103.806521,86.4,3.0,3.000000,1.000000,0.0,22.5,0.0,28.600,2022,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9538,9538,3C71BF164F90,860206,8602,86,10.579849,99.113146,71.5,66.0,6.964000,5.980000,0.0,180.0,3.4,33.250,2022,6,3
9539,9539,3C71BF18CEA4,490706,4907,49,16.494229,104.350891,64.7,62.0,1.000000,0.000000,0.0,0.0,0.0,34.700,2022,7,3
9540,9540,3C71BF164F90,860206,8602,86,10.579849,99.113146,86.5,0.0,6.964000,5.980000,0.0,180.0,1.3,27.725,2022,6,4
9541,9541,3C71BF18EA64,300903,3009,30,15.112831,102.052114,90.7,11.0,1.135135,0.135135,0.0,157.5,0.0,26.000,2022,7,2


In [46]:
unique_values = train['mac'].unique()
print(unique_values)

['3C71BF164F90' '3C71BF1B1E28' '30AEA4F7AE44' '807D3AF57920'
 '3C71BF18EA64' '3C71BF17CDBC' '3C71BF18CEA4' '3C71BF15DB04']


In [73]:
print(df['mac'].dtype)

object


In [47]:
tempmac = train.groupby('mac')['temp'].mean()
print("temp by mac:")
print(tempmac)

temp by mac:
mac
30AEA4F7AE44    31.634828
3C71BF15DB04    29.199262
3C71BF164F90    27.914598
3C71BF17CDBC    29.327918
3C71BF18CEA4    30.039069
3C71BF18EA64    30.137575
3C71BF1B1E28    28.138175
807D3AF57920    30.535940
Name: temp, dtype: float64


In [7]:
def mac_to_int(mc):
    # Ensure no delimiters (in case any) and convert to int from hex
    return int(mc, 16)

# Apply the transformation to the column
train['mac_numerical'] = train['mac'].apply(mac_to_int)

In [8]:
train.drop(['mac'], axis=1, inplace=True)

In [64]:
train

,id,tambon_code,amphur_code,province_code,latitude,longitude,humid,light,pm10,pm2.5,rainfall,wind_direct,wind_speed,temp,year,month,time_period,mac_numerical
0,0,860206,8602,86,10.579849,99.113146,89.9,5.0,6.000000,5.000000,0.0,180.0,6.1,26.925,2022,7,4,1
1,1,630601,6306,63,16.712990,98.573417,80.8,0.0,8.000000,7.000000,0.0,135.0,1.4,27.250,2022,7,1,1
2,2,102601,1026,10,13.777972,100.569662,60.8,51.0,2.799043,1.511962,0.0,270.0,3.9,34.075,2022,6,2,1
3,3,630601,6306,63,16.712990,98.573417,87.1,0.0,6.000000,5.000000,0.0,135.0,0.0,26.500,2022,6,1,1
4,4,380704,3807,38,18.142499,103.806521,86.4,3.0,3.000000,1.000000,0.0,22.5,0.0,28.600,2022,7,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9538,9538,860206,8602,86,10.579849,99.113146,71.5,66.0,6.964000,5.980000,0.0,180.0,3.4,33.250,2022,6,3,1
9539,9539,490706,4907,49,16.494229,104.350891,64.7,62.0,1.000000,0.000000,0.0,0.0,0.0,34.700,2022,7,3,1
9540,9540,860206,8602,86,10.579849,99.113146,86.5,0.0,6.964000,5.980000,0.0,180.0,1.3,27.725,2022,6,4,1
9541,9541,300903,3009,30,15.112831,102.052114,90.7,11.0,1.135135,0.135135,0.0,157.5,0.0,26.000,2022,7,2,1


In [77]:
unique_values_macneu = train['mac_numerical'].unique()
print(unique_values_macneu)

[ 66459234881424  66459235196456  53526650138180 141275348433184
  66459235052132  66459234979260  66459235045028  66459234851588]


In [13]:
from sklearn.model_selection import train_test_split

# Features and target variable
id_column = train['id']
X = train.drop(columns=['id', 'temp'],axis=1)
y = train['temp']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
X_train

,tambon_code,amphur_code,province_code,latitude,longitude,humid,light,pm10,pm2.5,rainfall,wind_direct,wind_speed,year,month,time_period,mac_numerical
5466,860206,8602,86,10.579849,99.113146,74.8,70.0,6.964000,5.980000,0.0,225.0,7.5,2022,7,3,66459234881424
4180,102601,1026,10,13.777972,100.569662,61.2,65.0,2.799043,1.511962,0.0,270.0,8.3,2022,6,3,53526650138180
5009,102601,1026,10,13.777972,100.569662,75.4,49.0,2.799043,1.511962,0.0,180.0,5.0,2022,6,2,53526650138180
81,330802,3308,33,14.533026,104.495172,71.3,47.0,5.000000,4.000000,0.0,247.5,1.6,2022,7,3,66459234851588
1323,380704,3807,38,18.142499,103.806521,68.7,80.0,5.000000,3.000000,0.0,45.0,1.5,2022,5,2,141275348433184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,380704,3807,38,18.142499,103.806521,85.8,0.0,2.000000,1.000000,0.0,135.0,3.4,2022,7,4,141275348433184
5191,490706,4907,49,16.494229,104.350891,96.7,0.0,1.000000,0.000000,0.0,315.0,0.0,2022,7,1,66459235045028
5390,300903,3009,30,15.112831,102.052114,94.5,0.0,1.135135,0.135135,0.0,0.0,0.0,2022,7,1,66459235052132
860,550404,5504,55,18.241106,100.690577,99.0,0.0,6.000000,4.000000,0.6,0.0,0.0,2022,7,2,66459234979260


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")


Mean Squared Error: 1.45
R-squared: 0.93


In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = -grid_search.best_score_

print("Best Parameters:", best_params)
print("Best MSE Score (lower is better):", best_score)


Fitting 5 folds for each of 108 candidates, totalling 540 fits


KeyboardInterrupt: 

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define the parameter distribution
param_dist = {
    'n_estimators': randint(50, 200),  # Randomly sample from this range
    'max_depth': [None] + list(range(10, 31, 5)),  # Include None and a range
    'min_samples_split': randint(2, 11),  # Randomly sample from 2 to 10
    'min_samples_leaf': randint(1, 5),  # Randomly sample from 1 to 4
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                   n_iter=100, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error',
                                   random_state=42)

# Fit the random search to the data
random_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = random_search.best_params_
best_score = -random_search.best_score_  # Negate because we used neg_mean_squared_error

print("Best Parameters:", best_params)
print("Best MSE Score (lower is better):", best_score)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)

# Prepare the submission DataFrame
submission = pd.DataFrame({
    'id': id_column.loc[X_test.index],  # Get the corresponding IDs
    'predicted_temp': y_pred
})